In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import joblib

In [11]:
import pandas as pd
df = pd.read_csv("module3.csv",encoding='latin-1')
df_pay = df["injection_type"]
df.drop(["payload"],axis=1,inplace=True)
df.drop(["injection_type"],axis=1,inplace=True)
X = df.values
y = df_pay.values

In [3]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X ,y , test_size=0.2,random_state= 2) 
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)
y_train = y_train.type(torch.LongTensor)
y_test = y_test.type(torch.LongTensor)


In [3]:

class BayesianNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BayesianNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
def train(model, optimizer, criterion, train_loader):
    model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    return train_loss / len(train_loader)

In [6]:
def evaluate(model, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()
    return test_loss / len(test_loader), correct / len(test_loader.dataset)

In [7]:
input_size = 9
hidden_size = 1024
output_size = 4
num_epochs = 15
batch_size = 512
learning_rate = 0.01

In [8]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [9]:
model = BayesianNN(input_size, hidden_size, output_size)

In [10]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [11]:
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, criterion, train_loader)
    test_loss, test_accuracy = evaluate(model, criterion, test_loader)
    print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1 - Train Loss: 0.8358, Test Loss: 0.1014, Test Accuracy: 0.9740
Epoch 2 - Train Loss: 0.1594, Test Loss: 0.1111, Test Accuracy: 0.9744
Epoch 3 - Train Loss: 0.1168, Test Loss: 0.0920, Test Accuracy: 0.9746
Epoch 4 - Train Loss: 0.1062, Test Loss: 0.0739, Test Accuracy: 0.9782
Epoch 5 - Train Loss: 0.0765, Test Loss: 0.0750, Test Accuracy: 0.9779
Epoch 6 - Train Loss: 0.0774, Test Loss: 0.0797, Test Accuracy: 0.9764
Epoch 7 - Train Loss: 0.0772, Test Loss: 0.1170, Test Accuracy: 0.9649
Epoch 8 - Train Loss: 0.0762, Test Loss: 0.0821, Test Accuracy: 0.9780
Epoch 9 - Train Loss: 0.0689, Test Loss: 0.0808, Test Accuracy: 0.9781
Epoch 10 - Train Loss: 0.0700, Test Loss: 0.0776, Test Accuracy: 0.9768
Epoch 11 - Train Loss: 0.0704, Test Loss: 0.0764, Test Accuracy: 0.9763
Epoch 12 - Train Loss: 0.0686, Test Loss: 0.0720, Test Accuracy: 0.9794
Epoch 13 - Train Loss: 0.0716, Test Loss: 0.0729, Test Accuracy: 0.9782
Epoch 14 - Train Loss: 0.0655, Test Loss: 0.0720, Test Accuracy: 0.9771
E

In [4]:
file_path = "C:/Users/kisho/Documents/GitHub/Risk_Quantification/Threat_Identification"

def dump(model , filename):
    with open(filename, "wb") as f:
        joblib.dump(model , f)

In [5]:
dump(model , filename=f"{file_path}/bnn.pkl")

NameError: name 'model' is not defined

In [6]:
import sys 
sys.path.append('/Users/kisho/Documents/GitHub/Risk_Quantification')
sys.path

from WAF_Model_Trainer.utils import extract_feature

In [7]:
payload_n = "&& perl -e 'print ""X""x4096'"
d = extract_feature(payload_n)
d.drop(columns=d.columns[0], axis=1,  inplace=True)
im_arr = np.array(d.values)
im_arr = im_arr.flatten()
im_arr




array([25,  0,  5,  5,  0,  0,  0,  0,  4], dtype=int64)

In [8]:
def predict(model, data):
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
    return predicted

In [9]:
new_data = torch.Tensor(im_arr)
bnn_model = joblib.load("bnn.pkl")

predicted_class = predict(bnn_model, new_data.unsqueeze(0))
if(predicted_class.item() == 0) :
    attack_type = "NORMAL"
elif(predicted_class.item() == 1) :
    attack_type = "COMMAND INJECTION"
elif(predicted_class.item() == 2) :
    attack_type = "SQL INJECTION"
else:
    attack_type = "CROSS-SITE SCRIPTING (XSS)"

print(attack_type) 

COMMAND INJECTION


In [13]:
df_attacks = pd.read_csv("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Logged_Output/waf_payloads.csv")
fin_arr = []


In [16]:
for index, row in df_attacks.iterrows():
    current_row = []
    current_payload = row['Malicious Payload']
    current_row.append(current_payload)
    d = extract_feature(current_payload)
    d.drop(columns=d.columns[0], axis=1,  inplace=True)
    im_arr = np.array(d.values)
    im_arr = im_arr.flatten()
    current_data = torch.Tensor(im_arr)
    predicted_class = predict(bnn_model, current_data.unsqueeze(0))
    if(predicted_class.item() == 0) :
        attack_type = "NORMAL"
    elif(predicted_class.item() == 1) :
        attack_type = "COMMAND INJECTION"
    elif(predicted_class.item() == 2) :
        attack_type = "SQL INJECTION"
    else:
        attack_type = "CROSS-SITE SCRIPTING (XSS)"
    current_row.append(attack_type)
    fin_arr.append(current_row)
    
fin_arr = np.array(fin_arr)

    

In [17]:
df = pd.DataFrame({'Payload': fin_arr[:, 0], 'Attack Type': fin_arr[:, 1]})
df.info
df.to_csv("C:/Users/kisho/Documents/GitHub/Risk_Quantification/Logged_Output/bnn_attack_types.csv")